In [22]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.backtester.base.base import rsi_below_entries, plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.utils.caching import clear_cache

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[40],
    prices=prices.close
)


In [23]:
df_array, order_records, df_cart_array = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(3,6,1),
    risk_rewards=np.arange(3,5,1),
    size_pct=1.,
    gains_pct_filter=40,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [24]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(10)    

,order_id,or_set,ind_set,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,106,0,31,242.86,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00",NaN,"10,416.00","12,651.14"
1,1,106,0,32,242.86,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00",NaN,"10,355.65","12,577.84"
2,2,106,0,33,242.86,"10,577.50","10,716.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,287.69","12,495.29"
3,3,106,0,34,242.86,"10,523.00","10,667.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,240.65","12,438.16"
4,4,106,0,370,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,555.06","12,438.16"
5,5,106,0,381,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,556.02","12,438.16"
6,6,106,0,388,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,557.97","12,438.16"
7,7,106,0,389,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,561.36","12,438.16"
8,8,106,0,405,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,563.30","12,438.16"
9,9,106,0,406,NaN,NaN,"10,667.34",NaN,MovedTSL,NaN,"1,000.00",NaN,"10,640.90","12,438.16"


In [25]:
pd.DataFrame(df_array).sort_values(by='gains_pct', ascending=False)


,or_set,ind_set,win_rate,gains_pct,to_the_upside,total_trades,total_BE,total_fees,total_pnl,ending_eq
9,161,0,50.79,66.72,0.04,63.00,0,117.69,667.20,"1,667.20"
1,107,0,53.97,63.47,0.14,63.00,0,94.67,634.73,"1,634.73"
8,160,0,47.69,60.47,0.04,65.00,0,115.97,604.72,"1,604.72"
0,106,0,48.48,50.89,0.05,66.00,0,92.39,508.87,"1,508.87"
4,146,0,46.32,45.90,0.07,95.00,0,189.00,458.96,"1,458.96"
7,159,0,43.48,44.52,0.01,69.00,0,115.44,445.16,"1,445.16"
6,158,0,55.71,43.54,0.02,70.00,0,108.59,435.36,"1,435.36"
3,128,0,56.34,42.94,0.01,71.00,0,108.94,429.39,"1,429.39"
2,119,0,46.32,42.09,0.04,95.00,0,184.81,420.95,"1,420.95"
5,155,0,56.34,41.78,0.02,71.00,0,109.73,417.80,"1,417.80"


In [26]:
df_cart = pd.DataFrame(df_cart_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2,3,4,5,6,7,8,9
order_settings_id,106,107,119,128,146,155,158,159,160,161
max_equity_risk_pct,4.00,4.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
risk_rewards,4.00,4.00,3.00,3.00,4.00,4.00,4.00,4.00,4.00,4.00
size_pct,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
tsl_based_on,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price
tsl_pcts_init,4.00,4.00,3.00,4.00,3.00,4.00,4.00,4.00,4.00,4.00
tsl_trail_by_pct,3.00,3.00,1.00,1.00,1.00,1.00,2.00,3.00,3.00,3.00
tsl_when_pct_from_avg_entry,2.00,3.00,3.00,3.00,3.00,3.00,3.00,1.00,2.00,3.00


In [27]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [28]:
clear_cache()